# NIRSpec IFU Data Reduction Pipeline

By the JWST ERS TEMPLATES team (B. Welch, T. Hutchison, +).   
Version: Nov. 2022. 

Based on a combination of notebooks from STSCI:  
- [MRS_FlightNB1](https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb)
- [spec_mode_stage_2](https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_2.ipynb)
- [spec_mode_stage_3](https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_3.ipynb)
- [jwebbinar5_nirspecifu](https://github.com/spacetelescope/jwebbinar_prep/blob/main/ifu_session/jwebbinar5_nirspecifu.ipynb)

Prerequisites: Install JWST pipeline. See TEMPLATES pipeline installation notebook ([linked here](https://github.com/JWST-Templates/Notebooks/blob/main/0_install_pipeline.ipynb)) for help.

------------

# Example Pipeline Run on NIRSpec IFU Data
Currently this is the re-observed IFU data on SGAS-1723 from the TEMPLATES data. We have set up this notebook such that you can specify which of the TEMPLATES targets you would like to use in the pipeline. 

### Important Note:
>This version is applying background subtraction in the Stage 3 pipeline. It takes the Stage 2 output extracted 1-D spectra ("x1d.fits") from the background observations, and creates an average spectrum to universally subtract from the science data. 
>
>There is another possibility for background subtraction, where the dedicated background exposures would be subtracted from the science exposures during the Stage 2 pipeline. These would be "image from image" subtractions, rather than creating a universal background spectrum. We ran into an issue with this type of background subtraction. The grating wheel positions were slightly different between the science and background observations. The stage 2 pipeline did not like that one bit, and refused to apply any background subtraction (the grating wheel position affects the wavelength solution, and thus would throw off this style of background subtraction). We may test the 2D background subtraction on another dataset, but the assumption is it will be noisier. 
>
>Also, the outlier_detection step in the Stage 3 pipeline is currently dysfunctional (as of pipeline version 1.8.3). We include a band-aid solution for now. Hopefully this issue is fixed in later pipeline versions.

#### Additional Note:
>The output printed by the various pipeline stages is extensive -- which is great for troubleshooting and learning the pipeline but also makes these notebooks very large in size (which GitHub hates).  We've added an `stpipe-log.cfg` file to this repository that will pipe all of that printed output into a log file called `pipeline-run.log`, which will be generated in the same working directory.  To help those running the pipeline from scratch in tracking down what happens in each stage, we've separated that log informtion into the three different pipeline stages, which can be found in: `pipeline-stage1.log`, `pipeline-stage2.log`, `pipeline-stage3.log` (when you run this pipeline yourself, all of this log info would appear in the `pipeline-run.log` file instead).

In [1]:
# ------
# PATHS: 
# -----

# 1) point to where you keep your data
# input_path = '/Users/bdwelch1/Documents/data/templates/sdss1723/full_uncal_data/' # for B. Welch
input_path = "/Users/tahutch1/data/raw/jwst/ers/templates/MAST_2022-10-18T1132/JWST/" # for T. Hutchison

# 2) point to where you want your processed outputs to live
# output_path = '/Users/bdwelch1/Documents/data/templates/sdss1723/pmap1009_ditherleak_1Dbg/' # for B. Welch
output_path = "/Users/tahutch1/data/raw/jwst/ers/templates/reduced/" # for T. Hutchison

# 3) Un-comment these 2 lines if you haven't specified these CRDS paths elsewhere
# -- (like described in the installation script linked at the top of this notebook)
# os.environ["CRDS_PATH"] = home + "crds_cache/jwst_ops"
# os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"


In [2]:
# general packgaes needed
import numpy as np
import glob
import os
import zipfile
import urllib.request
import json

# astronomy code
from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

In [3]:
# plotting packages
import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:
# The Stage 1 pipeline:
from jwst.pipeline import Detector1Pipeline

# The Stage 2 & 3 pipelines:
# -- calwebb_spec and spec3
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# data models
from jwst import datamodels

# association file utilities
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

# Setting up variables
First we'll look at all of the targets who we have data for, then we can specify the target want we want to reduce.

In [5]:
files = glob.glob(input_path + '*nrs*/*_uncal.fits') # list the uncalibrated (level 1b) files.
files = sorted(files)

target_list = []

# running through files, checking header for target name
for exposure in files:
    head = fits.getheader(exposure)
    
    # splitting the object name from data type (IFU vs SKY)
    splitting_name = head['TARGPROP'].split('-')
    target_list.append(splitting_name[0]) # just taking the object name

targets_list = list(set(target_list))

print('Pick one of these target names and assign it to the "target" variable below:\n\n', targets_list)


Pick one of these target names and assign it to the "target" variable below:

 ['SPT2147', 'SPT0418', 'SGAS1723']


In [6]:
target = 'SGAS1723' # specify your target name here, as a string

In [7]:
# checking that the file system is in place for our reduced data
# if not, creating the folders

targ_folder = target + '/'

folders = ['',                           # the main folder
           'L2a/','L2a/sci/','L2a/bkg/', # output from Stage 1
           'L2b/','L2b/sci/','L2b/bkg/', # output from Stage 2
           'L3/' ]                       # output from Stage 3

for folder in folders:
    path = output_path + targ_folder + folder
    if os.path.exists(path) == False: # if folder doesn't exist
        print('Creating folder ' + path)
        os.system('mkdir ' + path) # creates the folder
        

Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2a/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2a/sci/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2a/bkg/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2b/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2b/sci/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L2b/bkg/
Creating folder /Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/L3/


#### Assigning additional variables that we'll need in Stage 2 based upon the specified TEMPLATES target:

In [8]:
program_id = '01355' # 5-digit string. ID of observing program. '01355' for TEMPLATES


# below is a dictionary listing all of the observation+target-specific variables:
# values chosen based on target specified above.
#
# YOU DO NOT NEED TO EDIT THIS DICTIONARY FOR TEMPLATES SOURCES
obs_info = {'SGAS1723':{'obs_num_sci':'027',
                        'obs_num_bkg':'028',
                        'visit_num':'001',
                        'group_sci':['02','06'], # for SGAS1723: '02' is g140h data, '06' is g395h data
                        'group_leak':['04','08']} # associated dedicated leakcals
           # 
           # will be adding the values for the other 3 targets soon!
           }


In [9]:
# choosing the right dictionary of values based on chosen TEMPLATES target
target_obs_info = obs_info[target]

obs_num_sci = target_obs_info['obs_num_sci'] # 3-digit string. Observation number for science observations.
obs_num_bkg = target_obs_info['obs_num_bkg'] # 3-digit string. Observation number for background observations.

visit_num = target_obs_info['visit_num'] # 3-digit string. Visit number. '001' for most observations
group_sci = target_obs_info['group_sci'] # List of 2-digit strings. Visit group numbers for science observations.
group_leak = target_obs_info['group_leak'] # List of 2-digit strings. Visit group numbers for leakcals.


#### Updating output path to include target name

In [10]:
# printing original output path
print(output_path,end='\n\n')

# adding target name to the end of the output path
output_path += targ_folder

print(f'Updated path, to include chosen target: \n{output_path}')


/Users/tahutch1/data/raw/jwst/ers/templates/reduced/

Updated path, to include chosen target: 
/Users/tahutch1/data/raw/jwst/ers/templates/reduced/SGAS1723/


--------------------

## Stage 1 - Detector-level processing
The first time this runs it will be *glacially* slow, as many GB of reference files need to be downloaded.
Changing the maximum_cores variable in the parameter file can speed up this step. The default is "None", which uses one core. Other options are "quarter", "half", and "all", which will use 1/4, 1/2, or all available cores respectively.

To control parameters directly in the notebook, simply add `det1.[step].[param] = [val]` before calling `det1(exposure)`.
For example, to utilize multiple cores, add the line `det1.jump.maximum_cores = 'half'` and `det1.ramp_fit.maximum_cores = 'half'` to use half the available cores in those two steps.

The default values work well for the stage 1 pipeline. The only parameter that should be changed is the maximum_cores. "half" typically gives good performance without totally incapacitating your machine (assuming it has half physical and half virtual cores). "full" speeds things up a bit more in exchange for eating all of your computers processing power (only use this if you don't need your computer for anything else for several hours). The default option "None" uses one core, and is glacially slow. 

In [ ]:
# Run the pipeline, splitting outputs into "sci" and "bkg" output folders
# Leakcals will be in the same folders as their associated observations

for exposure in files: 
    det1 = Detector1Pipeline()
    
    # set output directory based on sci vs bkg exposure
    head = fits.open(exposure)[0].header
    if head['TARGPROP'] == f'{target}-IFU':
        det1.output_dir = output_path + 'L2a/sci'
    elif head['TARGPROP'] == f'{target}-SKY':
        det1.output_dir = output_path + 'L2a/bkg'
    else: print('not target')
    
    # now we set other parameters:
    det1.save_results = True
    det1.jump.maximum_cores = 'half'
    det1.ramp_fit.maximum_cores = 'half'

    # run stage 1
    det1(exposure)


------------------

## Stage 2 - Produce calibrated exposures
This stage uses the L2a outputs from the Stage 1 pipeline. We have named these "sci" for the science observations, and "bkg" for the background observations.  

#### Note:
>We have chosen to implement leak corrections (from dedicated leakcal exposures) for *both* the science and background observations. It doesn't seem to make too big of a difference (based on few short checks), but our sense is that it avoids double-subtracting light leaking through the MSA. 

To include leakcals (and background subtraction if desired) in the Stage 2 pipeline, an association file is needed. Below is a function to create these asn files from a combination of science, leak, and background exposures.

In [ ]:
# first, lets adapt the writel3asn function to produce what we want for the level 2 pipeline
def writel2asn(scifiles, leakfiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMSLevel2bBase, product_name=prodname)
    
    # Add leakcal files to the association
    if leakfiles:
        for ii in range(len(leakfiles)):
            asn['products'][0]['members'].append({'expname': leakfiles[ii], 'exptype': 'imprint'})
            
    # Add background files to the association
    if bgfiles:
        nbg=len(bgfiles)
        for ii in range(0,nbg):
            asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)
        

In this example, we implement the leakcal correction on a dither-by-dither basis, since each of our science and background dithers include corresponding leakcals. This seems to improve performance slightly (though for now this is likely confounded by changing reference files). Another option would be to use an average of all leakcal observations for this correction. 

Below is a block of code that creates the appropriate asn files for each exposure. This involves a lot of tedious file grouping, leading to the large and somewhat messy-looking code block. 

*We have included many comments to try to clarify the code as much as possible.*

In [ ]:

# "dither" leak_version will use each leakcal with its associated sci exposure
# "all" will average all leakcals and subtract the average from each sci exposure
leak_version = 'dither' # 'dither' or 'all'


# define some path names for easy access
level1_sci_dir = os.path.join(output_path, 'L2a/sci/')
level1_bg_dir = os.path.join(output_path, 'L2a/bkg/')
level2outputdir = os.path.join(output_path, 'L2b/')


# and combine some of these for convenience:
sci_base = 'jw' + program_id + obs_num_sci + visit_num + '_' # I always feel like
bkg_base = 'jw' + program_id + obs_num_bkg + visit_num + '_' # somebody's watching meeee


# specify science files
# currently assumes we use 2 gratings. Will need to edit for more/fewer grating observations
scifiles_g1 = sorted(glob.glob(level1_sci_dir + sci_base + group_sci[0] + '*rate.fits'))
scifiles_g2 = sorted(glob.glob(level1_sci_dir + sci_base + group_sci[1] + '*rate.fits'))

# and background files:
bgfiles_g1 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[0] + '*rate.fits'))
bgfiles_g2 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[1] + '*rate.fits'))


# collect leakcal files
sci_leakfiles_g1_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[0] + '*rate.fits'))
sci_leakfiles_g2_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[1] + '*rate.fits'))

bg_leakfiles_g1_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak[0] + '*rate.fits'))
bg_leakfiles_g2_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak[1] + '*rate.fits'))


# this version assumes you have the same number of dithers for 
# the science, leak, and background observations
if leak_version == 'dither':
    for i,file in enumerate(scifiles_g1):
        leak = sci_leakfiles_g1_all[i]
        expnum = file[-17:-10]
        outfile = 'jw'+program_id+'-o'+obs_num_sci+'_'+group_sci[0]+'101_spec2_'+expnum+'_asn.json' 
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], [leak], None, outfile, 'Level2')
        print(outfile)
        
    for i,file in enumerate(scifiles_g2):
        leak = sci_leakfiles_g2_all[i]
        expnum = file[-17:-10]
        outfile = 'jw'+program_id+'-o'+obs_num_sci+'_'+group_sci[1]+'101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], [leak], None, outfile, 'Level2')
        print(outfile)

    for i,file in enumerate(bgfiles_g1):
        leak = bg_leakfiles_g1_all[i]
        expnum = file[-17:-10]
        outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[0]+'101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], [leak], None, outfile, 'Level2')
        print(outfile)

    for i,file in enumerate(bgfiles_g2):
        leak = bg_leakfiles_g2_all[i]
        expnum = file[-17:-10]
        outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[1]+'101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], [leak], None, outfile, 'Level2')
        print(outfile)

print()

Below is some code to make `asn` files using all leakcals averaged for each science exposure. We are not currently maintaining this code (hence why it's commented out), but it was left in case it is helpful to anyone later.  

You are welcome to minimize this code cell by clicking on the blue line that appears on the left side of the cell (when you are active in the cell).

In [ ]:
# keeping this alternate version for posterity
# the below version will use average of all leakcals, and subtract that from each science exposure
'''
leakfiles_04_nrs1 = sorted(glob.glob(leakdir_sci + '/jw01355027001_04101_*nrs1_rate.fits'))
leakfiles_04_nrs2 = sorted(glob.glob(leakdir_sci + '/jw01355027001_04101_*nrs2_rate.fits'))
leakfiles_08_nrs1 = sorted(glob.glob(leakdir_sci + '/jw01355027001_08101_*nrs1_rate.fits'))
leakfiles_08_nrs2 = sorted(glob.glob(leakdir_sci + '/jw01355027001_08101_*nrs2_rate.fits'))
bg_leakfiles_04_nrs1 = sorted(glob.glob(leakdir_bg + '/jw01355028001_04101_*nrs1_rate.fits'))
bg_leakfiles_04_nrs2 = sorted(glob.glob(leakdir_bg + '/jw01355028001_04101_*nrs2_rate.fits'))
bg_leakfiles_08_nrs1 = sorted(glob.glob(leakdir_bg + '/jw01355028001_08101_*nrs1_rate.fits'))
bg_leakfiles_08_nrs2 = sorted(glob.glob(leakdir_bg + '/jw01355028001_08101_*nrs2_rate.fits'))


if leak_version == 'all':
    for file in scifiles_02:
        if "nrs1" in file:
            leak = leakfiles_04_nrs1
        elif "nrs2" in file:
            leak = leakfiles_04_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o027_02101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)

    for file in scifiles_06:
        if "nrs1" in file:
            leak = leakfiles_08_nrs1
        elif "nrs2" in file:
            leak = leakfiles_08_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o027_06101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)
        
    for file in bgfiles_02:
        if "nrs1" in file:
            leak = leakfiles_04_nrs1
        elif "nrs2" in file:
            leak = leakfiles_04_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o028_02101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)

    for file in bgfiles_06:
        if "nrs1" in file:
            leak = leakfiles_08_nrs1
        elif "nrs2" in file:
            leak = leakfiles_08_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o028_06101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)
'''

Now that we finished setting up the parameters for this stage, we can run it.

In [ ]:
# And now we run the pipeline with the asn files we just made! 

asnfiles_sci = glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_sci+'*asn.json')
asnfiles_bkg = glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_bkg+'*asn.json')

# run stage 2 on science frames
for asn in asnfiles_sci:
    spec2 = Spec2Pipeline()
    spec2.output_dir = output_path + 'L2b/sci'
    spec2.save_results = True
    spec2(asn)


# run stage 2 on background/sky frames
for asn in asnfiles_bkg:
    spec2 = Spec2Pipeline()
    spec2.output_dir = output_path + 'L2b/bkg/'
    spec2.save_results = True
    spec2(asn)


------------

## Stage 3 - Creating final data cubes
This stage compiles the individual calibrated exposures into a single, (theoretically) science-ready data cube. We first have to define an association file, which tells the pipeline which files to use as science exposures, and which files to use as backgrounds.

As mentioned above, for this iteration we are doing a "master" background subtraction at this stage. This creates a single 1D spectrum from an average of the dedicated background exposures (using the extracted 1D spectra from the previous stage). This averaged spectrum is then universally subtracted from the science data. 

#### NOTE!!!!
>#### outlier_detection is currently dysfuntional!!
>As of pipeline version 1.8.3, the outlier_detection step of the Stage 3 pipeline is performing poorly. Conversations with the helpdesk indicate that they are aware of the issue and are looking into solutions. For now, the recommendation is to simply skip this step. 
>
>The S1723 data are quite noisy, so some level of outlier detection is needed. We therefore include a band-aid solution, wherein we make custom cuts on the Stage 2 products (cal.fits files). These cuts need to be checked for each science target, but we find that brightness values between -1 and 20 MJy/sr work well enough for now. This interval removes the most egregious noise without sacrificing flux in the brightest emission lines, but some artifacts still appear in the final data cubes.

In [ ]:
# BELOW COPIED FROM DAVID LAW'S MIRI MRS NOTEBOOK: 
# https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb
# 
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    if bgfiles:
        nbg=len(bgfiles)
        for ii in range(0,nbg):
            asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

#### *UNTIL THE OUTLIER_DETECTION IS FIXED:*

>The `spec3.outlier_detection` step is currently broken (our team asked the help desk, and their response was "yeah don't use that right now"). So to get around this, we're going to apply some cuts to the level 2 data products to get rid of most of the obvious noise. This is far from perfect, but it will work as a band-aid solution until the actual outlier detection code is fixed. 

In [ ]:
# COURTESY OF D. LAW:
# outlier detection step is currently broken. 
# So instead, here we will manually remove outliers from the cal.fits files

from stcal import dqflags


def cut_cal(infile, outfile, max_threshold=20, min_threshold=-1):
    hdu=fits.open(infile)
    sci=hdu['SCI'].data
    dq=hdu['DQ'].data
    
    dnubit=dqflags.interpret_bit_flags('DO_NOT_USE', mnemonic_map=datamodels.dqflags.pixel)
    indx=np.where((dq & dnubit) != 0)
    sci[indx]=np.nan

    indx=np.where((sci > max_threshold) | (sci < min_threshold))
    sci[indx]=np.nan
    dq[indx] = np.bitwise_or(dq[indx], dnubit)
    
    hdu['SCI'].data=sci
    hdu.writeto(outfile, overwrite=True)
    
# fix this path later
orig_calfiles = glob.glob(level2outputdir + '/sci/*cal.fits') # cal files output from L2 pipeline

for file in orig_calfiles:
    outfile = file[:-5] + '2.fits'
    cut_cal(file, outfile, max_threshold=20, min_threshold=-1)
    

In [ ]:
# Make association files - use sci for data, bkgd for background
calfiles = glob.glob(output_path + 'L2b/sci/*cal2.fits')

bkgfiles = glob.glob(output_path + 'L2b/bkg/*x1d.fits')

asnfile = os.path.join(output_path, 'L3/L3asn.json')

writel3asn(calfiles, bkgfiles, asnfile, 'Level3')

In [ ]:
# Next step, call stage 3 pipeline on new asn file
asn = os.path.join(output_path, 'L3/L3asn.json')

# setting it up this way (rather than with "call") gives a bit more flexibility to edit parameters on the fly
spec3 = Spec3Pipeline()
spec3.output_dir = output_path + 'L3/'

# Outlier detection is broken, so we skip it for now
spec3.outlier_detection.skip = True 
spec3.save_results = True # DON'T FORGET THIS OR YOU'LL WASTE SEVERAL HOURS FOR NAUGHT!

# run stage 3
spec3.run(asn)


## Congrats, you're done!